# Class function for QuanQual, UnivariateTable, replace_outliers, FreqTable

In [25]:
import pandas as pd
import numpy as np
#---univariate Class----
class myunivariate():
    #QuanQual list split
    def QuanQual(df):
        quan=[x for x in df if df[x].dtype!=object]
        qual=[x for x in df if df[x].dtype==object]
        return quan,qual
    #--------------------------------------------------
    #create New Table for central Tendency & descriptive
   #create New Table for central Tendency & descriptive#create New Table for central Tendency & descriptive
    def UnivariateTable(df,num_cals):
        import pandas as pd
        import numpy as np
        descriptive=pd.DataFrame(index=["Mean","Median","Mode",
                                        "Min","Q1-25%","Q2-50%","Q3-75%","Max",
                                       "IQR","1.5-Rule","Lower-Bound","Upper-Bound",
                                       "Skew","Kurtosis",
                                       "Variance","Std"]
                                 ,columns=num_cals)
        for cal in num_cals:
            #print(cal)
            descriptive.loc["Mean", cal] = round(df[cal].mean(), 2)
            descriptive.loc["Median", cal] = df[cal].median()
            descriptive.loc["Mode", cal] = round(df[cal].mode()[0], 2)
            descriptive.loc["Min",cal]=df.describe().loc['min',cal]
            descriptive.loc["Q1-25%",cal]=df.describe().loc['25%',cal]
            descriptive.loc["Q2-50%",cal]=df.describe().loc['50%',cal]
            descriptive.loc["Q3-75%",cal]=df.describe().loc['75%',cal]
            descriptive.loc["Max",cal]=df.describe().loc['max',cal]
            descriptive.loc["IQR",cal]=descriptive.loc["Q3-75%",cal]-descriptive.loc["Q1-25%",cal]
            descriptive.loc["1.5-Rule",cal]=1.5*descriptive.loc["IQR",cal]
            descriptive.loc["Lower-Bound",cal]=descriptive.loc["Q1-25%",cal]-descriptive.loc["1.5-Rule",cal]
            descriptive.loc["Upper-Bound",cal]=descriptive.loc["Q3-75%",cal]+descriptive.loc["1.5-Rule",cal]
            descriptive.loc["Skew",cal]=round(descriptive[cal].skew(), 2)
            descriptive.loc["Kurtosis",cal]=round(descriptive[cal].kurtosis(), 2)
            descriptive.loc["Variance",cal]=round(descriptive[cal].var(), 2)
            descriptive.loc["Std",cal]=round(descriptive[cal].std(), 2)
        
        #return descriptive
        
        print("\n-------------- Outlier info --------------\n")
        lesser=[]
        greater=[]
        for cal in num_cals:
            if descriptive.loc["Min",cal]<descriptive.loc["Lower-Bound",cal]:
                lesser.append(cal)
                print(f"Lower outlier in :'{cal}' value :", descriptive.loc["Min",cal])
            if descriptive.loc["Max",cal]>descriptive.loc["Upper-Bound",cal]:
                greater.append(cal)
                print(f"Upper outlier in :'{cal}' value :", descriptive.loc["Max",cal])
        print("Lesser outlier :",lesser,"\nGreater outlier:",greater)
        print("\n-------------- Univariate Table  --------------\n")
        return descriptive       
    #---------------------------------------------------
    #---------------------------------------------------
    # Function to replace outliers using np.percentile
    def replace_outliers(df):
        print("\n-------------- Outliers Replaced info --------------\n")
        for col in df.select_dtypes(exclude="object").columns:
            # Q1 (25th percentile) and Q3 (75th percentile)
            #dropna() removes missing values (NaN) before calculating, 
            #Q1 = df[col].quantile(0.25) # other way in panda calculate Q1, Q2
            #Q3 = df[col].quantile(0.75)
            Q1 = np.percentile(df[col].dropna(), 25)
            Q3 = np.percentile(df[col].dropna(), 75)
            IQR = Q3 - Q1
    
            # Bounds
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
    
            # Replace lower outliers with lower bound
            df.loc[df[col] < lower_bound, col] = lower_bound
            
            # Replace higher outliers with upper bound
            df.loc[df[col] > upper_bound, col] = upper_bound
            print(f"{col}: Outliers replaced → lower < {lower_bound:.2f}, upper > {upper_bound:.2f}")
        
        return df
    #---------------------------------------------------
    #create Function for freqtable
    def FreqTable(df,column_name):
        print(f"\n---- Frequency Table for '{column_name}'------\n")
        freqtable = df[column_name].value_counts().reset_index()
        # Rename columns
        freqtable.columns = ["Unique_Values", "Frequency"]
        # Calculate relative frequency
        freqtable["Relative_Frequency"] = freqtable["Frequency"] / freqtable["Frequency"].sum()
        # Cumulative relative frequency
        freqtable["Cumulative_Rel_Freq"] = freqtable["Relative_Frequency"].cumsum()
        # Optional: sort by Frequency
        #freqtable = freqtable.sort_values(by="Frequency").reset_index(drop=True)
        return freqtable 
    

# Call Class and Function 

In [26]:
#from my_class_univariate import myunivariate
import pandas as pd
import numpy as np
dataset = pd.read_csv("Placement.csv")

# Get quantitative and qualitative columns
quan, qual = myunivariate.QuanQual(dataset)
quan.remove("sl_no")
print(quan)

['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']


In [27]:
# Descriptive statistics with outliers
desc = myunivariate.UnivariateTable(dataset,quan)
print(desc)


-------------- Outlier info --------------

Lower outlier in :'hsc_p' value : 37.0
Upper outlier in :'hsc_p' value : 97.7
Upper outlier in :'degree_p' value : 91.0
Upper outlier in :'salary' value : 940000.0
Lesser outlier : ['hsc_p'] 
Greater outlier: ['hsc_p', 'degree_p', 'salary']

-------------- Univariate Table  --------------

              ssc_p    hsc_p degree_p  etest_p   mba_p              salary
Mean           67.3    66.33    66.37     72.1   62.28           288655.41
Median         67.0     65.0     66.0     71.0    62.0            265000.0
Mode           62.0     63.0     65.0     60.0    56.7            300000.0
Min           40.89     37.0     50.0     50.0   51.21            200000.0
Q1-25%         60.6     60.9     61.0     60.0  57.945            240000.0
Q2-50%         67.0     65.0     66.0     71.0    62.0            265000.0
Q3-75%         75.7     73.0     72.0     83.5  66.255            300000.0
Max            89.4     97.7     91.0     98.0   77.89          

In [28]:
# Replace outliers
cleaned_dataset = myunivariate.replace_outliers(dataset)


-------------- Outliers Replaced info --------------

sl_no: Outliers replaced → lower < -106.00, upper > 322.00
ssc_p: Outliers replaced → lower < 37.95, upper > 98.35
hsc_p: Outliers replaced → lower < 42.75, upper > 91.15
degree_p: Outliers replaced → lower < 44.50, upper > 88.50
etest_p: Outliers replaced → lower < 24.75, upper > 118.75
mba_p: Outliers replaced → lower < 45.48, upper > 78.72
salary: Outliers replaced → lower < 150000.00, upper > 390000.00


In [29]:
# Descriptive statistics without outlier
desc = myunivariate.UnivariateTable(cleaned_dataset,quan)
print(desc)


-------------- Outlier info --------------

Lesser outlier : [] 
Greater outlier: []

-------------- Univariate Table  --------------

              ssc_p   hsc_p degree_p  etest_p   mba_p         salary
Mean           67.3   66.33    66.36     72.1   62.28      277648.65
Median         67.0    65.0     66.0     71.0    62.0       265000.0
Mode           62.0    63.0     65.0     60.0    56.7       300000.0
Min           40.89   42.75     50.0     50.0   51.21       200000.0
Q1-25%         60.6    60.9     61.0     60.0  57.945       240000.0
Q2-50%         67.0    65.0     66.0     71.0    62.0       265000.0
Q3-75%         75.7    73.0     72.0     83.5  66.255       300000.0
Max            89.4   91.15     88.5     98.0   77.89       390000.0
IQR            15.1    12.1     11.0     23.5    8.31        60000.0
1.5-Rule      22.65   18.15     16.5    35.25  12.465        90000.0
Lower-Bound   37.95   42.75     44.5    24.75   45.48       150000.0
Upper-Bound   98.35   91.15     88.5

In [32]:
# Frequency table for all Quan
#freq = myunivariate.FreqTable(cleaned_dataset,"ssc_p")
#print(freq)
for x in quan:
    freq = myunivariate.FreqTable(cleaned_dataset,x)
    print(freq)